# `virtual`: How to virtualize Redset

In [1]:
%reload_ext autoreload
%autoreload 2

import virtual

s3_parquet_filepath = 's3://redshift-downloads/redset/serverless/full.parquet'

### One-Shot table compression

In [2]:
# Virtualize Parquet file stored on S3.
virtual.to_format(s3_parquet_filepath, 'redset_virtual.parquet', model_types=['sparse-lr'], prefix='demo-debug/')

Running schema inference..
Drilling functions..
[{'name': 'instance_id', 'type': 'INTEGER'}, {'name': 'cluster_size', 'type': 'INTEGER'}, {'name': 'user_id', 'type': 'BIGINT'}, {'name': 'database_id', 'type': 'BIGINT'}, {'name': 'query_id', 'type': 'BIGINT'}, {'name': 'arrival_timestamp', 'type': 'TIMESTAMP'}, {'name': 'compile_duration_ms', 'type': 'DOUBLE'}, {'name': 'queue_duration_ms', 'type': 'BIGINT'}, {'name': 'execution_duration_ms', 'type': 'BIGINT'}, {'name': 'feature_fingerprint', 'type': 'VARCHAR'}, {'name': 'was_aborted', 'type': 'INTEGER'}, {'name': 'was_cached', 'type': 'INTEGER'}, {'name': 'cache_source_query_id', 'type': 'INTEGER'}, {'name': 'query_type', 'type': 'VARCHAR'}, {'name': 'num_permanent_tables_accessed', 'type': 'DOUBLE'}, {'name': 'num_external_tables_accessed', 'type': 'DOUBLE'}, {'name': 'num_system_tables_accessed', 'type': 'DOUBLE'}, {'name': 'read_table_ids', 'type': 'VARCHAR'}, {'name': 'write_table_ids', 'type': 'VARCHAR'}, {'name': 'mbytes_scanned'

### Compare to vanilla `Parquet`

In [3]:
import os
import demo_util
%pip install requests

print(f'Redset parquet size: {demo_util.get_public_s3_file_size(s3_parquet_filepath) / 1_000_000} MB')
print(f'Redset virtual size: {os.path.getsize('redset_virtual.parquet') / 1_000_000} MB')

Note: you may need to restart the kernel to use updated packages.
Redset parquet size: 322.619676 MB
Redset virtual size: 314.702683 MB


### Query the vanilla Parquet file

In [4]:
import duckdb

duckdb.sql(f'select num_scans, count(*) from read_parquet("{s3_parquet_filepath}") group by num_scans order by num_scans;').fetchdf()

,num_scans,count_star()
0,0,4992205
1,1,2191086
2,2,910544
3,3,37527
4,4,33871
...,...,...
123,226,63
124,246,4
125,261,3
126,262,22


### Query the virtualized Parquet file

In [5]:
virtual.query(
  'select num_scans, count(*) from read_parquet("redset_virtual.parquet") group by num_scans order by num_scans',
  engine = 'duckdb'
)

,"COALESCE(num_scans_outlier, (CAST(round((((((((((((0.0024 * instance_id) + (-0.0003 * user_id)) + (-0.1212 * database_id)) + (0.0754 * was_aborted)) + (0.2777 * was_cached)) + (-0.0114 * num_permanent_tables_accessed)) + (-0.0937 * num_external_tables_accessed)) + (-0.1652 * num_system_tables_accessed)) + (0.4646 * num_joins)) + (0.4077 * num_aggregations)) + -0.4392), 0) AS BIGINT) + num_scans_offset))",count_star()
0,0,4992205
1,1,2191086
2,2,910544
3,3,37527
4,4,33871
...,...,...
123,226,63
124,246,4
125,261,3
126,262,22
